In [2]:
import numpy as np
%load_ext Cython

In [85]:
"""
Most of the BJ stuff should follow general game conventions. 
We initially model more intricate details (burning, shuffling protocol) based on local casino.
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
"""

# configurations
NDECKS: int = 8
# NPLAYERS: int = 4 # affects how many cards you may be able to count
DECK_PENETRATION: int = 2 # typical number of decks dealt out before reshuffle

In [86]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings as our dtype so we can have a Cython memview over the numpy card shoe of C char type
shoe = shoe.astype('S2')

numcardsdeal = len(deck) * DECK_PENETRATION

In [68]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

8.2 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [69]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards, values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

65.2 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

In [11]:
strat

,0,1,2,3,4,5,6,7,8,9,10
0,HARD:,2,3,4,5,6,7,8,9,T,A
1,4,H,H,H,H,H,H,H,H,H,H
2,5,H,H,H,H,H,H,H,H,H,H
3,6,H,H,H,H,H,H,H,H,H,H
4,7,H,H,H,H,H,H,H,H,H,H
5,8,H,H,H,H,H,H,H,H,H,H
6,9,H,DH,DH,DH,DH,H,H,H,H,H
7,10,DH,DH,DH,DH,DH,DH,DH,DH,H,H
8,11,DH,DH,DH,DH,DH,DH,DH,DH,DH,DH
9,12,H,H,S,S,S,H,H,H,H,H


In [73]:
%%cython 
import numpy as np
cimport cython 

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# C uses ascii integer representation for characters
# Numeric characters in our shoe will have an ascii value less than ascii of 'a'
# English characters will have an ascii value greater than ascii of 'a'
cdef int ASCII_A = 65 

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(card_dtype_struct[:] shoe not None, int numcardsdeal):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 
    """
    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef int = count # card count
    
    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
    while dealt < numcardsdeal: 
        

    if shoe[0].face == b'5': 
        return 5
    else:
        return 3





In [72]:
shoe[:50]

array([b'5D', b'JH', b'7S', b'8C', b'3D', b'2H', b'KH', b'JS', b'AD',
       b'8D', b'8C', b'AD', b'5S', b'2D', b'5C', b'QD', b'8H', b'4C',
       b'QS', b'2S', b'KD', b'KD', b'3S', b'TH', b'2D', b'9S', b'9D',
       b'4C', b'2D', b'AH', b'7C', b'AH', b'7H', b'5D', b'7C', b'TH',
       b'QD', b'KC', b'4H', b'JH', b'9C', b'2S', b'JH', b'2C', b'TC',
       b'3H', b'JD', b'4D', b'JH', b'JC'], dtype='|S2')

In [74]:
play_shoe(shoe, numcardsdeal)

5